In [1]:
import tensorflow as tf

In [ ]:
x_train, x_valid = None
y_train, y_valid = None

# 使用顺序API创建模型
tf.random.set_seed(42)
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# 编译模型
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# 训练模型
model.fit(x_train, y_train, epochs=5, validation_data=(x_valid, y_valid))

In [ ]:
x_train_wide = None
x_train_deep = None
x_valid_wide = None
x_valid_deep = None

# 使用函数式API创建模型
input_wide = tf.keras.layers.Input(shape=[5])
input_deep = tf.keras.layers.Input(shape=[6])
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden_1 = tf.keras.layers.Dense(30, activation='relu')(norm_deep)
hidden_2 = tf.keras.layers.Dense(3, activation='relu')(hidden_1)
concat = tf.keras.layers.Concatenate()([norm_wide, hidden_2])
output = tf.keras.layers.Dense(1, activation='sigmoid')(concat)
aux_output = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_2)
model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])
# 编译模型:每个输出都需要单独编译
optimiser = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss=['mse', 'mse'],
              loss_weights=[0.9, 0.1],
              optimizer=optimiser,
              metrics=['RootMeanSquaredError'])
# 训练模型
norm_layer_wide.adapt(x_train_wide)
norm_layer_deep.adapt(x_train_deep)
model.fit({'input_wide':x_train_wide, 'input_deep':x_train_deep}, (y_train, y_train),
          epochs=20,
          validation_data=([x_valid_wide, x_valid_deep], (y_valid, y_valid)))

In [ ]:
# 使用子类化API创建模型
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.2)


class WideAndDeepModel(tf.keras.Model):
    def __init__(self, units=30, activation=leaky_relu, **kwargs):
        super().__init__(**kwargs)
        self.norm_layer_wide = tf.keras.layers.Normalization()
        self.norm_layer_deep = tf.keras.layers.Normalization()
        self.hidden1 = tf.keras.layers.Dense(units, activation=activation)
        self.hidden2 = tf.keras.layers.Dense(units, activation=activation)
        self.main_output = tf.keras.layers.Dense(1, activation='sigmoid')
        self.aux_output = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        input_wide, input_deep = inputs
        norm_wide = self.norm_layer_wide(input_wide)
        norm_deep = self.norm_layer_deep(input_deep)
        hidden1 = self.hidden1(norm_deep)
        hidden2 = self.hidden2(hidden1)
        concat = tf.keras.layers.Concatenate()([norm_wide, hidden2])
        output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return output, aux_output
    

model = WideAndDeepModel(30, 'relu', name='wide_and_deep_model')

In [ ]:
# 批量归一化
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(300, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(10, activation='softmax')
])